In [ ]:
#Conectar al drive local
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Conectar al repositorio
import sys
import os
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #para importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #para que suba y descargue archivos de la ruta del repositorio de trabajo

In [1]:
import sqlite3 as sql #para crear y trabajar bases de datos ligeras
import pandas as pd
import numpy as np

In [ ]:
con = sql.connect('data/db_movies.db')